# Import

In [1]:
import bootstats as boot
import numpy as np
import os

First we generate random normal data as the input. The `Bootstrapper` class needs numpy arrays of shape `NVars x NConfigs`.

# Create Bootstrapper instances

In [2]:
NVars1   = 128   
NVars2   = 4   
NConfigs = 2000    # Number of HMC samples for each observable

data = np.random.normal(size=[NVars1, NVars2, NConfigs])

One can either create a `Bootstrapper` instance from bootstrap parameters or from already generated bootstrap indices.

In [3]:
NSamples = 1000
NBinSize = 5
NBins    = int(NConfigs / NBinSize)
NSize    = NBins 

## Generate the instance from parameters

In [4]:
bs = boot.Bootstrapper(
    data,               # The input data
    NSamples=NSamples,  # The number of bootstrap samples -> N_B
    NSize=NSize,        # The size of each bootstrap samples -> N_S
    NBinSize=NBinSize   # The size of bins before bootstrapping -> N_{BS}
)
bs

Bootstrapper(NSamples=1000, NSize=400, NBinSize=5, NConfigs=2000, NVars=512, NBins=400)

This initializes the bootstrapper class which
1. Bins the initial data according to `NBinSize` for each variable:
    `bs.data[i] = mean(data[i:i+NBinSize])`, where the beginning is chopped of in case `NConfigs % NBinSize != 0`.
2. Generates random indices according to a uniform distribution `U(0, NBins-1)` of shape `NSamples x NSize`.

In [5]:
print("Indices shape:", bs.indices.shape)
print("Indices range is range(NBins):", (np.unique(bs.indices) == np.arange(NBins)).all())

Indices shape: (1000, 400)
Indices range is range(NBins): True


## Generate instance from indices

Similarly, one can also create an instance by passing indices to the class. This becomes handy in case one wants to reproduce results.

In [6]:
bs2 = boot.Bootstrapper(
    data,               # The input data
    NBinSize=NBinSize,  # The size of bins before bootstrapping -> N_{BS}
    indices=bs.indices  # Bootstrap indices for accesing the data
)

In [7]:
print("Both instances have the same data, parameters and indices:", bs == bs2)

Both instances have the same data, parameters and indices: True


# Computation of bootstrap samples

In order to compute the distribtution of means, one has to compute the mean of the binned data over each individual bootstrap sample: `bootData[nVar, nSample] = mean(binnedData[nVar, index[nSample, 0:NSize]])`. This information is stored in the `samples` member.

In [8]:
print("C++ result - numpy result = {:1.3e}".format(
    np.std(bs.samples - np.mean(bs.data[:, :, bs.indices], axis=bs.data.ndim))
))

C++ result - numpy result = 0.000e+00


Note that `bs.data` is not in general equal to the input data because of the binning...

Also, because this can be quite intensive to compute (also in `C++`), the `Bootstrapper` class just computes this information when accessed (it calls `bs._getSamples()` once). Therefore, only the first computation takes time.

# Test against numpy

In [9]:
%timeit bs._getSamples()

464 ms ± 8.16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%timeit np.mean(bs.data[:, :, bs.indices], axis=bs.data.ndim)

1.91 s ± 63.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# HDF5 IO

In [11]:
fileName  = "test.h5"
groupName = "testGroup"
bs.exportHDF5(fileName, groupName, writeSamples=True)

In [12]:
bs3 = boot.Bootstrapper(data, h5Info={"fileName":fileName, "groupName":groupName})

In [13]:
bs3 == bs

True

In [14]:
os.remove(fileName)